In [8]:
import json
import math
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import namedtuple, OrderedDict

In [9]:
sys.path.insert(0, '../src/lib')

In [10]:
from utils.geometry import create_camera_matrix, euler_angles_to_rotation_matrix, euler_angles_to_quaternions, quaternion_to_euler_angle, quaternion_upper_hemispher
from utils.image import affine_transform, get_affine_transform, draw_msra_gaussian, draw_umich_gaussian, gaussian_radius, pad_img_sides
from utils.kaggle_cars_utils import parse_annot_str

In [11]:
INPUT_PATH = '../data/pku-autonomous-driving'

In [12]:
def read_split_file(split, data_dir=INPUT_PATH):
    path = os.path.join(data_dir, 'split', split+'.txt')
    with open(path) as f:
        return [line.rstrip('\n')[:-4] for line in f]


def load_annotations(df, img_ids):
    anns = []
    for img_id in img_ids:
        for car in gen_img_annotation(img_id, df):
            anns.append(car['rotation'])
    return anns


def gen_img_annotation(img_id, df):
    cond = df['ImageId'] == img_id
    cars = df.loc[cond, 'PredictionString'].values[0]
    cars = parse_annot_str(str(cars))
    return cars

In [13]:
df = pd.read_csv(os.path.join(INPUT_PATH, 'train.csv'))
img_ids = read_split_file('train') + read_split_file('val_462')
rotations = load_annotations(df, img_ids)
len(img_ids), len(rotations)

(4262, 49684)

In [14]:
bad_match_idx = []
eq_thresh = 2e-7

for i, rot0 in enumerate(rotations):
    q = euler_angles_to_quaternions(rot0)[0]
    q = quaternion_upper_hemispher(q)
    rot1 = quaternion_to_euler_angle(q)
    
    rot_diff = rot1 - rot0
    
    if (rot_diff > eq_thresh).any():
        bad_match_idx.append(i)
        
len(bad_match_idx)

0